In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import numpy as np
import ast
import utils
from data_loader import YelpDataLoader
from sqlalchemy import create_engine

In [10]:
from constants import STANFORD, CAMBRIDGE, VANCOUVER

In [11]:
cambridge_loader = YelpDataLoader('cambridge', 10)

SELECT * FROM businesses WHERE distance_to_cambridge <= 10

            SELECT 
                b.business_id,
                name,
                address,
                city,
                state,
                b.stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses as b
            LEFT JOIN tips 
                ON b.business_id = tips.business_id
            WHERE distance_to_cambridge <= 10
            


In [15]:
cambridge_loader.df_close_tips.text

0         Best massage ever!  Thanks Brian for the recom...
1         Please don't waste your money at this school. ...
2                                                      Scam
3         Hallucinatory man always sitting and talking a...
4           Subway has moved into the old Dairy Queen spot.
                                ...                        
180975    first time try this lovely place... just love ...
180976                             good price！good quality！
180977                                      simply the best
180978                       wow!!!!!!!! all i have to say.
180979                                                 None
Name: text, Length: 180980, dtype: object

In [13]:
utils.filter_df_with_categories(cambridge_loader.df_close_tips, 'Bubble Tea')

,business_id,name,address,city,state,stars,distance_to_cambridge,attributes,categories,hours,text
9936,bwZlXQ3FPFWmewJupcOcAQ,Tsaocaa,10A Tyler St,Boston,MA,3.5,2.964133,"{'BusinessAcceptsCreditCards': 'True', 'NoiseL...","[Bubble Tea, Juice Bars & Smoothies, Restaur...","{'Monday': '12:0-20:30', 'Tuesday': '12:0-20:3...",Pretty good!
9937,bwZlXQ3FPFWmewJupcOcAQ,Tsaocaa,10A Tyler St,Boston,MA,3.5,2.964133,"{'BusinessAcceptsCreditCards': 'True', 'NoiseL...","[Bubble Tea, Juice Bars & Smoothies, Restaur...","{'Monday': '12:0-20:30', 'Tuesday': '12:0-20:3...",They have a $10 credit card minimum!
26025,tiYqu1I671KdSRljLpxBXg,Sweet Heart,386 Trapelo Rd,Belmont,MA,4.0,3.943361,"{'HasTV': 'True', 'Ambience': '{'touristy': Fa...","[Bubble Tea, Cafes, Ice Cream & Frozen Yogur...","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21...","Great milk tea, friendly service"
26026,tiYqu1I671KdSRljLpxBXg,Sweet Heart,386 Trapelo Rd,Belmont,MA,4.0,3.943361,"{'HasTV': 'True', 'Ambience': '{'touristy': Fa...","[Bubble Tea, Cafes, Ice Cream & Frozen Yogur...","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21...",Mango is my absolute favorite will be going he...
26235,6ZE-iodwXRIA7rPOoM4lfQ,Pholicious,1201 Broadway,Saugus,MA,3.5,7.481981,"{'RestaurantsDelivery': 'False', 'RestaurantsA...","[Bubble Tea, Vietnamese, Restaurants, Food]","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",Great food and service. Especially for food co...
...,...,...,...,...,...,...,...,...,...,...,...
170885,v972-kNc67HyGqdpqoCHSQ,Gong Cha,154 Harvard Ave,Allston,MA,3.5,1.879708,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","[Bubble Tea, Food, Taiwanese, Restaurants]","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",Bubble tea was good but you have to pay separa...
170886,v972-kNc67HyGqdpqoCHSQ,Gong Cha,154 Harvard Ave,Allston,MA,3.5,1.879708,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","[Bubble Tea, Food, Taiwanese, Restaurants]","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",Get the strawberry flavored milk foam! It's wo...
170887,v972-kNc67HyGqdpqoCHSQ,Gong Cha,154 Harvard Ave,Allston,MA,3.5,1.879708,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","[Bubble Tea, Food, Taiwanese, Restaurants]","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",Good
170888,v972-kNc67HyGqdpqoCHSQ,Gong Cha,154 Harvard Ave,Allston,MA,3.5,1.879708,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","[Bubble Tea, Food, Taiwanese, Restaurants]","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",Subscribe to Gong Cha's loyalty program by giv...


In [24]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""SELECT *
            FROM tips
            """
df_tips = pd.read_sql(
    query,
    con=engine,
)

CPU times: user 4.06 s, sys: 226 ms, total: 4.28 s
Wall time: 4.29 s


In [42]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /Users/justin.xu/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /Users/justin.xu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/justin.xu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/justin.xu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/justin.xu/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/justin.xu/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [58]:
from textblob import TextBlob
tip = TextBlob(df_tips.text[0])
tip, tip.sentiment

(TextBlob("Carne asada chips..."), Sentiment(polarity=0.0, subjectivity=0.0))

In [94]:
df_tips_ = df_tips[:1000]

utils.insert_sentiment_column(df_tips_)

/Users/justin.xu/restaurant_planner/utils.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["polarity"], df["subjectivity"] = zip(


,index,user_id,business_id,text,date,compliment_count,polarity,subjectivity
0,0,WCjg0jdHXMlwbqS9tZUx8Q,ENwBByjpoa5Gg7tKgxqwLg,Carne asada chips...,2011-07-22 19:07:35.000000,0,0.000000,0.000000
1,1,42-Z02y9bABShAGZhuSzrQ,jKO4Og6ucdX2-YCTKQVYjg,Best happy hour from 3pm to 6pm! $1 off martin...,2014-09-10 07:33:29.000000,0,0.725000,0.650000
2,2,5u7E3LYp_3eB8dLuUBazXQ,9Bto7mky640ocgezVKSfVg,"Nice people, skilled staff, clean location - b...",2013-12-13 23:23:41.000000,0,0.519444,0.733333
3,3,wDWoMG5N9oI4DJ-p7z8EBg,XWFjKtRGZ9khRGtGg2ZvaA,"1/2-price bowling & the ""Very"" Old Fashion are...",2017-07-11 23:07:16.000000,0,0.565000,0.630000
4,4,JmuFlorjjRshHTKzTwNtgg,mkrx0VhSMU3p3uhyJGCoWA,"Solid gold's. Great sauna. Great staff, too. E...",2016-11-30 08:46:36.000000,0,0.600000,0.533333
...,...,...,...,...,...,...,...,...
995,995,q_IZV7WmBcrwYp4x8nioog,QFpNhUUD5cSPqJAVa4fANA,Bring somebody here for the great view.,2016-12-01 18:37:24.000000,0,0.800000,0.750000
996,996,pqY60Dj4xCkVfJhuI23J3Q,-MrP6EY3zgXQoMvUD3TAng,Indulgent beef brisket for lunch - supremely d...,2012-05-31 19:26:07.000000,0,1.000000,1.000000
997,997,bFT7y7vTG1z63IyA7fsWsg,VznOFJQbZ9PZKrzdXkkiiA,instant seating great beer lovely shade on the...,2015-06-06 20:19:37.000000,0,0.433333,0.722222
998,998,UKHa4BDvcIvdVoiJsYYbnw,hwfF4tuCVBlFXY-K1pjeDA,Great cuisine!! I was here for brunch!!! P.S: ...,2017-10-19 20:52:09.000000,0,0.400000,0.538889


In [93]:
df_tips_100

,index,user_id,business_id,text,date,compliment_count,polarity,subjectivity
0,0,WCjg0jdHXMlwbqS9tZUx8Q,ENwBByjpoa5Gg7tKgxqwLg,Carne asada chips...,2011-07-22 19:07:35.000000,0,0.000000,0.000000
1,1,42-Z02y9bABShAGZhuSzrQ,jKO4Og6ucdX2-YCTKQVYjg,Best happy hour from 3pm to 6pm! $1 off martin...,2014-09-10 07:33:29.000000,0,0.725000,0.650000
2,2,5u7E3LYp_3eB8dLuUBazXQ,9Bto7mky640ocgezVKSfVg,"Nice people, skilled staff, clean location - b...",2013-12-13 23:23:41.000000,0,0.519444,0.733333
3,3,wDWoMG5N9oI4DJ-p7z8EBg,XWFjKtRGZ9khRGtGg2ZvaA,"1/2-price bowling & the ""Very"" Old Fashion are...",2017-07-11 23:07:16.000000,0,0.565000,0.630000
4,4,JmuFlorjjRshHTKzTwNtgg,mkrx0VhSMU3p3uhyJGCoWA,"Solid gold's. Great sauna. Great staff, too. E...",2016-11-30 08:46:36.000000,0,0.600000,0.533333
...,...,...,...,...,...,...,...,...
95,95,bwotC8HIBIh9URtnDGNmiw,M1pN60ruY204Fx3z37ZxPw,Chicken Teriyaki is yummy yummy!!,2011-08-01 16:43:02.000000,0,-0.937500,0.950000
96,96,4NasTqV-SeT76ijXUt-nXQ,q7w-JIWeEp64H2tou4sTrw,WTF froyo,2010-07-20 01:57:22.000000,0,-0.500000,1.000000
97,97,T_wXXKfNHNz4rcGuZDt4ZQ,I_ahD8eAsSUv2I6gO3l1fA,The service here is absolutely horrible so mak...,2014-06-15 14:44:17.000000,0,-0.012222,0.681111
98,98,GUs6qYi8j7CKjgdpVRY7Ig,dsqmIQtNB21J3g-56mVmsg,They have great salads and margaritas.,2014-08-24 00:01:29.000000,0,0.800000,0.750000


In [90]:
%%time
df_tips_100 = df_tips[:100]
# df_tips_100['polarity'], df_tips_100['subjectivity'] = df_tips_100.text.apply(lambda x: TextBlob(x).sentiment).str
# df_tips_100.loc[:,('polarity','subjectivity')] = df_tips_100.text.apply(lambda x: TextBlob(x).sentiment).str
df_tips_100['polarity'], df_tips_100['subjectivity'] = zip(*df_tips_100.text.apply(lambda x: TextBlob(x).sentiment))


CPU times: user 26.5 ms, sys: 1.24 ms, total: 27.7 ms
Wall time: 26.6 ms


<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
df_tips_100

,index,user_id,business_id,text,date,compliment_count,polarity,subjectivity
0,0,WCjg0jdHXMlwbqS9tZUx8Q,ENwBByjpoa5Gg7tKgxqwLg,Carne asada chips...,2011-07-22 19:07:35.000000,0,0.000000,0.000000
1,1,42-Z02y9bABShAGZhuSzrQ,jKO4Og6ucdX2-YCTKQVYjg,Best happy hour from 3pm to 6pm! $1 off martin...,2014-09-10 07:33:29.000000,0,0.725000,0.650000
2,2,5u7E3LYp_3eB8dLuUBazXQ,9Bto7mky640ocgezVKSfVg,"Nice people, skilled staff, clean location - b...",2013-12-13 23:23:41.000000,0,0.519444,0.733333
3,3,wDWoMG5N9oI4DJ-p7z8EBg,XWFjKtRGZ9khRGtGg2ZvaA,"1/2-price bowling & the ""Very"" Old Fashion are...",2017-07-11 23:07:16.000000,0,0.565000,0.630000
4,4,JmuFlorjjRshHTKzTwNtgg,mkrx0VhSMU3p3uhyJGCoWA,"Solid gold's. Great sauna. Great staff, too. E...",2016-11-30 08:46:36.000000,0,0.600000,0.533333
...,...,...,...,...,...,...,...,...
95,95,bwotC8HIBIh9URtnDGNmiw,M1pN60ruY204Fx3z37ZxPw,Chicken Teriyaki is yummy yummy!!,2011-08-01 16:43:02.000000,0,-0.937500,0.950000
96,96,4NasTqV-SeT76ijXUt-nXQ,q7w-JIWeEp64H2tou4sTrw,WTF froyo,2010-07-20 01:57:22.000000,0,-0.500000,1.000000
97,97,T_wXXKfNHNz4rcGuZDt4ZQ,I_ahD8eAsSUv2I6gO3l1fA,The service here is absolutely horrible so mak...,2014-06-15 14:44:17.000000,0,-0.012222,0.681111
98,98,GUs6qYi8j7CKjgdpVRY7Ig,dsqmIQtNB21J3g-56mVmsg,They have great salads and margaritas.,2014-08-24 00:01:29.000000,0,0.800000,0.750000


In [32]:
df_tips.sample(20).iloc[:,:-2]

,index,user_id,business_id,text
203662,203662,hoQ4uH-tZAJgCOWx479clA,xyVO973jt1bRk_3IpnYzzA,Open until 11 p.m.
724986,724986,zY6HKLdPPRD-YK8d8QDSNA,OfgCuP2QN165d-EsrCCIYg,Decor is pleasant. Food smells great. Hope it ...
497586,497586,0G-bcpdR48tfZy6koGUiTQ,_WNv5lBB95rOOWG_IlqkRg,"Quick stop!! No, seriously !"
884461,884461,nKs6D0d5eDLfdPOLEp5CyQ,7sHSPMpHntVe1XiTRZT47w,Will come back. I just moved here from Albuqu...
132283,132283,qLe0OnA9Cgj5TM2_e3js7Q,QzFBu9Zj5zS-sWyrmysEOQ,I need food late at night. This is the place t...
62269,62269,_w-ebdXf59sdH7f8u6JIXA,jBUspc1MPbwGtBav_CDX1w,The #21 For 2 is perfect and a good deal.
1027669,1027669,BDZjT5At0WVbDQInXZnCxg,_sVJWD5I0LfAYsLQPq0xew,Let's go Krogering.......
250142,250142,Rkw5_mQM47QA2qbGABfYgw,ZCJC2gmjnKZgbhm5gQY7tw,Going to HEB a few hours before the Super Bowl...
1032029,1032029,PhTNcLr8uRMKtfPTo1KqaQ,uBMU1bMebMySm8BZk4fw6g,:)))
797469,797469,gx5j0ylWlevoBWikRa9M6A,wV9lUJ9ZkfkF3FEjSJ52UA,Not to bad


In [17]:
%%time
engine = create_engine("sqlite:///yelp.db", echo=False)
query = f"""SELECT 
                b.business_id,
                name,
                address,
                city,
                state,
                b.stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses as b
            LEFT JOIN tips 
                ON b.business_id = tips.business_id
            WHERE distance_to_cambridge <= 10
            AND categories LIKE "%Bubble Tea%"
            """
df = pd.read_sql(
    query,
    con=engine,
)

CPU times: user 3.09 s, sys: 1.8 s, total: 4.88 s
Wall time: 4.92 s


In [21]:
df.text.apply(lambda x: x.count(' '))

0       11
1        0
2        0
3        4
4        0
        ..
1940    11
1941     3
1942    11
1943     5
1944     3
Name: text, Length: 1945, dtype: int64

In [5]:
%%time
df_close = utils.load_close_businesses('cambridge', 10)

SELECT * FROM businesses WHERE distance_to_cambridge <= 10
CPU times: user 1.58 s, sys: 67.5 ms, total: 1.65 s
Wall time: 1.65 s


In [7]:
%%time
df_close_tips = utils.load_close_tips('cambridge', 10)


            SELECT 
                b.business_id,
                name,
                address,
                city,
                state,
                b.stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses as b
            LEFT JOIN tips 
                ON b.business_id = tips.business_id
            WHERE distance_to_cambridge <= 10
            
CPU times: user 17.7 s, sys: 2.14 s, total: 19.8 s
Wall time: 20 s


In [45]:
%%time
df_close_reviews = utils.load_close_reviews('cambridge', 10)


            SELECT 
                businesses.business_id,
                name,
                address,
                city,
                state,
                stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses 
            LEFT JOIN reviews 
                ON businesses.business_id = reviews.business_id
            WHERE distance_to_cambridge <= 10
            


OperationalError: (sqlite3.OperationalError) ambiguous column name: stars
[SQL: 
            SELECT 
                businesses.business_id,
                name,
                address,
                city,
                state,
                stars,
                distance_to_cambridge,
                attributes,
                categories,
                hours,
                text
            FROM businesses 
            LEFT JOIN reviews 
                ON businesses.business_id = reviews.business_id
            WHERE distance_to_cambridge <= 10
            ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [369]:
_to_exclude = ['review_id', 'postal_code', 'date', 
               'attributes', 'address', 'business_id', 
               'categories', 'user_id', 'stars_x', 'stars_y',
              'latitude', 'longitude', 'is_open'
              'cool', 'funny']
cols = list(set(df_reviews_boba.columns) - set(_to_exclude))
df_reviews_boba[cols].sample(10)

,name,city,is_open,distance_to_cambridge,state,review_count,hours,useful,text,cool
1635935,Gong Cha,Allston,1,1.879708,MA,105,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",0,"I ordered a Strawberry Matcha Latte with boba,...",0
515715,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",2,Bought a drink here and after taking one sip i...,0
1593439,Monkey King Tea,Malden,1,3.670842,MA,244,"{'Wednesday': '11:30-20:0', 'Thursday': '11:30...",17,This place is too bomb! I fell in love right w...,10
515583,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",0,ChaTime is my staple boba tea place to go when...,0
948907,Dolphin Bay,Boston,1,1.756488,MA,486,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",0,This is probably my favorite Taiwanese restaur...,0
994068,Tealosophy,Boston,0,1.974531,MA,95,"{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",0,Good place to have snow shaved ice! They are r...,0
788366,Reign Drink Lab,Dorchester,1,5.456567,MA,104,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-20:0', '...",1,We LOVE Reign. Everything is consistently abso...,1
515647,Chatime,Boston,0,2.899948,MA,236,"{'Tuesday': '12:0-18:0', 'Wednesday': '12:0-18...",5,"Legit thick, creamy real taro milk tea! None o...",1
980955,Happy Lemon Boston,Boston,1,2.800257,MA,157,"{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'...",0,"So nice taste, and it's my favorite milk tea i...",0
762928,Gong Cha,Malden,1,4.090146,MA,27,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,"Saw this place was new and open as I drove by,...",0


In [372]:
df_boba_tips.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'distance_to_cambridge', 'user_id',
       'text', 'date', 'compliment_count'],
      dtype='object')

In [376]:
_to_exclude = ['postal_code', 'date', 'hours',
               'attributes', 'address', 'business_id', 
               'categories', 'user_id', 'stars_x', 'stars_y',
              'latitude', 'longitude', 'is_open'
               'funny']
cols = list(set(df_boba_tips.columns) - set(_to_exclude))
df_boba_tips[cols].sample(10)

,name,stars,city,is_open,distance_to_cambridge,state,review_count,compliment_count,text
153,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Yummy food for reasonable price!!!
244,Moon Flower House,4.5,Boston,1,1.793114,MA,121,0,很好吃！环境也很干净舒服，终于找到奥尔良烤翅太开心了吧。还是那个熟悉的配方，照烧鸡腿堡，新奥...
205,Happy Lemon Boston,4.0,Boston,1,2.800257,MA,157,0,"The check in is 10% off, not $2 off"
28,Chatime,3.0,Boston,0,2.899948,MA,236,0,It is officially opened! There is a buy one ge...
96,The Juice Bar,4.0,Boston,0,2.910175,MA,159,0,Love the jasmine tea!!
50,Chatime,3.0,Boston,0,2.899948,MA,236,0,Accepts credit card--no minimum.
172,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Authentic Taiwanese food! Reasonable price and...
146,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Kicking ass MTF awesome Taiwanese food with th...
6,Pholicious,3.5,Saugus,0,7.481981,MA,21,0,"If you don't want bubbles in your bubble tea, ..."
154,Dolphin Bay,4.5,Boston,1,1.756488,MA,486,0,Rice dishes are mediocre and over-priced given...
